# Lesson 02 Pandasを使ってみよう2

### panda-plyのインストールが必要です。
1. コマンドプロンプトで
```
pip install pandas-ply
```

2. ctl-Cで止めて、jupyter notebook を再起動する
3. ブラウザーを一度閉じてから立ち上げる
----
[pandas-ply/README.rst at master · coursera/pandas-ply](https://github.com/coursera/pandas-ply/blob/master/README.rst)   
[pandas-plyを使う - Qiita](http://qiita.com/gg_hatano/items/be4d3e7e969997e40381)  
[Pythonでのデータ操作 - Pandas_plyrを使ってみる - Qiita](http://qiita.com/hik0107/items/3dd260d9939a5e61c4f6)  

In [ ]:
import pandas as pd
from pandas_ply import install_ply, X, sym_call

install_ply(pd)

In [ ]:
import pandas as pd
csv_file_name = 'data/WA_Fn-UseC_-HR-Employee-Attrition.csv'

df = pd.read_csv(csv_file_name)
df.head()

## 1. 列を選ぶ (列名を変更する、条件により値を決めた列を作成する)
### ply_select
### (例) 年齢が40以上の場合はTrueとし、それ以外の場合はFalseとする

In [ ]:
df2 = df.ply_select("Department", "Age",
                Distance = X.DistanceFromHome,  ## カラム名を変更できる
                is_adult = (X.Age >= 40)  ## 新しいカラムを定義することも可能になる
                )
df2

### (例) 列名を変え、値を100分の1にし、先頭の10行にする。ピリオド(.)で続ける

In [ ]:
(df
    .ply_select(
      EducationField=X.EducationField,
      DailyRate_x100 = X.DailyRate / 100
      )
    .head(10)
    )

## 2. 列をまとめて、平均値や計数値を得る
### groupby

### (例) 平均値はmean()、計数値はsize()

In [ ]:
dataSummarize = (
    df
    .groupby('Department')
    .ply_select(
      meanAge=X.Age.mean(),
      candidateNum=X.Age.size(),
      )
    )
dataSummarize

## 3. 行を選ぶ (条件に合う行だけを選ぶ)
### ply_where

In [ ]:
df.ply_where(X.Age>40, 
               X.BusinessTravel == "Travel_Rarely",
               X.EducationField == "Life Sciences"
               )  #全ての条件にAndで満たすデータだけが選択される

In [ ]:
## under 30 
(df
    .ply_where(X.Age < 30)
    .head(10)
    )

## 4. 列名を変更する
[pandas DataFrameの行名・列名の変更 | nkmk log](https://nkmk.github.io/blog/python-pandas-dataframe-rename/)

In [ ]:
df.rename(columns={'Age': '年齢', 'Attrition': '自然減', 'BusinessTravel': '出張', 'DailyRate': '日当', \
        'Department': '部署', 'DistanceFromHome': '通勤距離', 'Education': '教育', 'EducationField': '教育領域'}, \
        index={'ONE': 'one'}, inplace=True)
df.head()

## 5. 日本語フォントが使用できるかを確認する

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set(font='IPAexGothic')
sns.plt.plot([0,1], [0,1]);  sns.plt.title('tofu - 豆腐')

### 上のグラフで「豆腐」の文字が見えない場合
* [日本語フォントの設定のまとめ](日本語フォントの設定のまとめ.ipynb) を実行します。

## 6. 簡単なグラフを描く

In [ ]:
dataSummarize = (
    df
    .groupby('部署')
    .ply_select(
      平均通勤距離=X.通勤距離.mean(),
      #candidateNum=X.年齢.size(),
      )
    )
dataSummarize

In [ ]:
import matplotlib.pyplot as plt
dataSummarize.plot()
plt.show()

## 7. ピボットテーブルを試してみる
[pandasでピボットテーブルを扱う · For myself tomorrow](https://tigawa.github.io/blog/2016/07/16/pivot/)

In [ ]:
pt = pd.pivot_table(df,
            # 集計したい縦のキー
               index=['部署','出張'],

            # 集計したい横のキー(複数指定可)
               columns='教育領域',

            # 集計したい項目 (指定がなければ、上記のキーになっていない項目)
               values='EmployeeCount',

            # 個数をカウントする。これがないとValuesの平均値になる。
               aggfunc=lambda x : len(x),

            # # NaN を 0埋めする
               fill_value = 0,
            )
pt

In [ ]:
pt.plot()
plt.show()

In [ ]:
pt = pd.pivot_table(df,
            # 集計したい縦のキー
               index=['部署','出張'],

            # 集計したい横のキー(複数指定可)
               columns=['教育領域', '自然減'],

            # 集計したい項目 (指定がなければ、上記のキーになっていない項目)
               values='EmployeeCount',

            # 個数をカウントする。これがないとValuesの平均値になる。
               aggfunc=lambda x : len(x),

            # # NaN を 0埋めする
               fill_value = 0,
            )
pt

## 8. 行の順序を整える
1. 名称順にする
    * 正順
        * df.reindex(index = natsorted(df.index))
    * 逆順にする
        * df.reindex(index = reversed(natsorted(df.index)))
1. 指定順にする
    1. 順序を得る
        * olist = df.index
     2. olistの要素を並べ変える
     3. df.reindex(index = olist)
----
* [python - Naturally sorting Pandas DataFrame - Stack Overflow](https://stackoverflow.com/questions/29580978/naturally-sorting-pandas-dataframe)

In [ ]:
pt

In [ ]:
pt.index

In [ ]:
from natsort import natsorted
#pt.sort_index(ascending=True, inplace=True)
pt.reindex(index=reversed(natsorted(pt.index)))

In [ ]:
index3 = natsorted(pt.index, reverse=True)
index3

In [ ]:
index3 = [('Sales', 'Travel_Rarely'),
 ('Sales', 'Travel_Frequently'),
 ('Sales', 'Non-Travel'),
 ('Human Resources', 'Travel_Rarely'),
 ('Human Resources', 'Travel_Frequently'),
 ('Human Resources', 'Non-Travel'),
 ('Research & Development', 'Travel_Rarely'),
 ('Research & Development', 'Travel_Frequently'),
 ('Research & Development', 'Non-Travel')]

In [ ]:
ptx = pt.reindex(index=index3)
ptx

In [ ]:
import seaborn as sns
sns.set(font='IPAexGothic')
#ptx = pt.reindex(index=natsorted(pt.index, reverse=True))
ptx = pt.reindex(index=index3)
ptx.plot(kind='barh', stacked=False)
plt.show()

In [ ]:
import seaborn as sns
sns.set(font='IPAexGothic')
#ptx = pt.reindex(index=natsorted(pt.index, reverse=True))
ptx = pt.reindex(index=index3)
ptx.plot(kind='barh', stacked=True)
plt.show()

In [ ]:
df_age = df.sort_values(by=["年齢"], ascending=True)
df_age

In [ ]:
dataSummarize

In [ ]:
dataSummarize.plot(kind='barh', legend=False)

In [ ]:
dataSummarize_r = dataSummarize.sort_values(by=["平均通勤距離"], ascending=False)
dataSummarize_r

In [ ]:
dataSummarize_r.plot(kind='barh', legend=False)